<a href="https://colab.research.google.com/github/farinazzo4/INFO3-BD2/blob/main/PrismaEd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
connection = sqlite3.connect("prisma.sqlite")

query = """
create Table Turma(
  Codigo varchar(10) not null  primary key,
  Curso varchar(20) not null,
  Periodo int not null
);
"""
execute_query(connection, query)

query = """
create Table Aluno(
  RA varchar(7) not null,
  CodigoTurma varchar(10) not null,
  Nome text not null,
  DataNasc date not null,

  primary key(RA, CodigoTurma),
  foreign key(CodigoTurma) references Turma(Codigo) on delete restrict
);
"""
execute_query(connection, query)

query = """
create Table ContatoDoAluno(
  RAAluno varchar(7) not null primary key,
  contato text not null,

  foreign key(RAAluno) references Aluno(RA) on delete cascade
);
"""
execute_query(connection, query)

query = """
create Table Professor(
  ID varchar(8) not null primary key
);
"""
execute_query(connection, query)

query = """
create Table ContatoDoProfessor(
  IDProfessor varchar(8) not null primary key,
  contato text not null,

  foreign key(IDProfessor) references Professor(ID) on delete cascade
);
"""
execute_query(connection, query)

query = """
create Table ProfessorLecionaEmTurma(
  IDProfessor varchar(8) not null,
  CodigoTurma varchar(10) not null,
  Disciplina text not null,

  primary key(IDProfessor, CodigoTurma),
  foreign key(IDProfessor) references Professor(ID) on delete restrict,
  foreign key(CodigoTurma) references Turma(Codigo) on delete restrict
);
"""
execute_query(connection, query)

query = """
create Table Questionario(
  NumeroRef integer not null primary key autoincrement
);
"""
execute_query(connection, query)

query = """
create Table AlunoRespondeQuestionario(
  RAAluno varchar(7) not null,
  NumeroRefQuestionario integer not null,
  DataResposta date not null,

  primary key(RAAluno, NumeroRefQuestionario),
  foreign key(RAAluno) references Aluno(RA) on delete cascade,
  foreign key(NumeroRefQuestionario) references Questionario(NumeroRef) on delete restrict
);
"""
execute_query(connection, query)

query = "INSERT INTO Turma(Codigo, Curso, Periodo) VALUES ('0123456789', 'Informatica','Primeiro Ano');"
execute_query(connection, query)

query = "INSERT INTO Aluno(RA, CodigoTurma, Nome, DataNasc) VALUES ('0123456', '0123456789', 'Maria Lúcia da Silva Rodrigues', '2006-09-17');"
execute_query(connection, query)

query = "INSERT INTO ContatoDoAluno(RAAluno, contato) VALUES ('0123456','nomedoaluno@gmail.com');"
execute_query(connection, query)

query = "INSERT INTO Professor(ID) VALUES ('6543210');"
execute_query(connection, query)

query = "INSERT INTO ContatoDoProfessor(IDProfessor, contato) VALUES ('6543210','nomeprofessor@gmail.com');"
execute_query(connection, query)

query = "INSERT INTO  ProfessorLecionaEmTurma(IDProfessor, CodigoTurma, Disciplina) VALUES ('6543210','0123456789', 'Banco de Dados');"
execute_query(connection, query)


NameError: name 'sqlite3' is not defined

-- ESCREVA AQUI O SELECT 1

In [ ]:
SELECT
    T.Codigo,
    T.Curso,
    COUNT(A.RA) AS TotalAlunos
FROM
    Turma T
LEFT JOIN
    Aluno A ON T.Codigo = A.CodigoTurma
GROUP BY
    T.Codigo, T.Curso;

-- ESCREVA AQUI A CONSULTA 2

In [ ]:
SELECT
    P.ID,
    C.contato,
    COUNT(PLT.CodigoTurma) AS QuantidadeDeTurmas
FROM
    Professor P
JOIN
    ProfessorLecionaEmTurma PLT ON P.ID = PLT.IDProfessor
JOIN
    ContatoDoProfessor C ON P.ID = C.IDProfessor
GROUP BY
    P.ID, C.contato
HAVING
    COUNT(PLT.CodigoTurma) > 1;

-- ESCREVA AQUI A CONSULTA COM 3 ENTIDADES

In [ ]:
SELECT
    A.Nome AS NomeDoAluno,
    A.RA,
    T.Curso,
    T.Periodo
FROM
    Aluno A
JOIN
    Turma T ON A.CodigoTurma = T.Codigo
JOIN
    ProfessorLecionaEmTurma PLT ON T.Codigo = PLT.CodigoTurma
WHERE
    PLT.IDProfessor = '6543210';

-- ESCREVA ABAIXO A VIEW

In [ ]:
CREATE VIEW VW_DetalhesAlunos AS
SELECT
    A.RA,
    A.Nome,
    A.DataNasc,
    T.Codigo AS CodigoTurma,
    T.Curso,
    T.Periodo,
    C.contato
FROM
    Aluno A
JOIN
    Turma T ON A.CodigoTurma = T.Codigo
LEFT JOIN
    ContatoDoAluno C ON A.RA = C.RAAluno;


-- ESCREVA ABAIXO O SELECT QUE RETORNA TODOS OS ELEMENTOS DA VIEW
SELECT * FROM VW_DetalhesAlunos;

## ESCREVA ABAIXO O "PROCEDURE"

In [ ]:
def matricular_aluno(conexao, ra, nome, data_nasc, codigo_turma):
    """
    Executa a lógica de matricular um novo aluno de forma segura e transacional.
    """
    cursor = conexao.cursor()
    try:
        # 1. Verificar se a turma existe
        cursor.execute("SELECT Codigo FROM Turma WHERE Codigo = ?", (codigo_turma,))
        if cursor.fetchone() is None:
            raise ValueError(f"Erro: A turma com código '{codigo_turma}' não foi encontrada.")

        # 2. Verificar se o RA já existe
        cursor.execute("SELECT RA FROM Aluno WHERE RA = ?", (ra,))
        if cursor.fetchone() is not None:
            raise ValueError(f"Erro: O RA '{ra}' já está cadastrado.")

        # 3. Inserir o novo aluno
        sql_insert = "INSERT INTO Aluno (RA, CodigoTurma, Nome, DataNasc) VALUES (?, ?, ?, ?)"
        cursor.execute(sql_insert, (ra, codigo_turma, nome, data_nasc))

        # Confirma a transação
        conexao.commit()
        print(f"Sucesso: Aluno '{nome}' (RA: {ra}) matriculado na turma '{codigo_turma}'.")

    except Exception as e:
        # Desfaz a transação em caso de erro
        conexao.rollback()
        print(f"Falha na matrícula: {e}")
    finally:
        pass

## ESCREVA ABAIXO UMA EXECUÇÃO DA PROCEDURE

In [ ]:
# --- Exemplo 1: Matrícula com Sucesso ---
print("--- Tentativa 1: Matrícula válida ---")
matricular_aluno(connection, '9876543', 'Carlos Souza', '2007-05-20', '0123456789')
print("-" * 20)


# --- Exemplo 2: Tentativa de matricular em turma que não existe ---
print("--- Tentativa 2: Turma inválida ---")
matricular_aluno(connection, '1122334', 'Ana Pereira', '2006-11-30', 'TURMA_INEXISTENTE')
print("-" * 20)


# --- Exemplo 3: Tentativa de matricular com RA duplicado ---
print("--- Tentativa 3: RA duplicado ---")
# Usando o mesmo RA do aluno inserido no início do notebook
matricular_aluno(connection, '0123456', 'Outro Aluno', '2005-01-15', '0123456789')
print("-" * 20)

## ESCREVA ABAIXO A FUNCTION

In [ ]:
import datetime

def CalcularIdade(data_nasc_str):
    """
    Calcula a idade a partir de uma data de nascimento em formato de string ('YYYY-MM-DD').
    Retorna a idade como um inteiro.
    """
    try:
        data_nasc = datetime.datetime.strptime(data_nasc_str, '%Y-%m-%d').date()
        hoje = datetime.date.today()
        idade = hoje.year - data_nasc.year - ((hoje.month, hoje.day) < (data_nasc.month, data_nasc.day))
        return idade
    except (ValueError, TypeError):
        # Retorna None se a data for inválida ou nula
        return None

connection.create_function("CalcularIdade", 1, CalcularIdade)


## ESCREVA ABAIXO UMA CONSULTA QUE INCLUI A FUNCTION

# Agora podemos usar a função 'CalcularIdade' diretamente no SQL.
query_com_funcao = """
SELECT
    Nome,
    DataNasc,
    CalcularIdade(DataNasc) AS Idade
FROM
    Aluno;
"""

-- ATIVIDADE 6

In [ ]:
-- PRIMEIRO, PRECISAMOS DA TABELA DE LOG. VAMOS CRIÁ-LA:
CREATE TABLE IF NOT EXISTS LogAlunoExcluido (
    IDLog INTEGER PRIMARY KEY AUTOINCREMENT,
    RAAluno VARCHAR(7) NOT NULL,
    NomeAluno TEXT,
    DataHoraExclusao DATETIME NOT- NULL
);


-- ## ESCREVA ABAIXO O TRIGGER
CREATE TRIGGER TRG_AuditoriaExclusaoAluno
AFTER DELETE ON Aluno
FOR EACH ROW
BEGIN
    INSERT INTO LogAlunoExcluido (RAAluno, NomeAluno, DataHoraExclusao)
    VALUES (OLD.RA, OLD.Nome, DATETIME('now', 'localtime'));
END;


-- ## ESCREVA ABAIXO UM COMANDO SQL QUE DISPARA O TRIGGER

-- Para este exemplo, vamos primeiro inserir um aluno para poder deletá-lo em seguida.
INSERT INTO Aluno(RA, CodigoTurma, Nome, DataNasc) VALUES ('7778889', '0123456789', 'Joana Pires', '2006-03-15');

-- Este é o comando que efetivamente dispara o trigger:
DELETE FROM Aluno WHERE RA = '7778889';

-- Para verificar o resultado, você pode consultar a tabela de log:
-- SELECT * FROM LogAlunoExcluido;